In [1]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

In [27]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

In [4]:

K = len(set(y_train))

i = Input(shape=x_train[0].shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)

model = Model(i, x)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0     

In [5]:
model.compile(optimizer='adam',
			loss='sparse_categorical_crossentropy',
			metrics=['accuracy'])


In [7]:
with tf.device("gpu:0"):
    r = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=5)


Epoch 1/5
1563/1563 [==============================] - 30s 12ms/step - loss: 1.3357 - accuracy: 0.5404 - val_loss: 1.0408 - val_accuracy: 0.6337
Epoch 2/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8598 - accuracy: 0.7018 - val_loss: 0.8740 - val_accuracy: 0.7071
Epoch 3/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.6985 - accuracy: 0.7603 - val_loss: 0.8503 - val_accuracy: 0.7214
Epoch 4/5
1563/1563 [==============================] - 19s 12ms/step - loss: 0.5829 - accuracy: 0.8001 - val_loss: 0.6364 - val_accuracy: 0.7848
Epoch 5/5
1563/1563 [==============================] - 18s 12ms/step - loss: 0.4952 - accuracy: 0.8297 - val_loss: 0.6540 - val_accuracy: 0.7862


In [10]:
predictions = np.argmax(model.predict(x_test), axis=-1)
conf_matrix = confusion_matrix(y_test, predictions)
conf_matrix

313/313 [==============================] - 2s 4ms/step


array([[827,  27,  37,  28,  16,   8,  17,  10,  19,  11],
       [  8, 930,   5,   3,   5,   3,  12,   1,   6,  27],
       [ 62,   3, 617,  40,  91,  68,  88,  28,   2,   1],
       [ 11,   4,  40, 576,  55, 170, 109,  28,   1,   6],
       [  8,   3,  29,  34, 804,  22,  69,  29,   2,   0],
       [  2,   4,  29,  87,  38, 753,  44,  42,   0,   1],
       [  3,   0,  13,  23,  17,  13, 925,   4,   1,   1],
       [  6,   1,  15,  32,  57,  43,  11, 833,   1,   1],
       [ 54,  51,  13,  38,   7,   5,  20,   7, 782,  23],
       [ 29,  98,   6,   6,   4,   4,  21,  12,   5, 815]], dtype=int64)

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

In [33]:
def enhance_image(image):
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpened = cv2.filter2D(image, -1, kernel)
    denoised = cv2.fastNlMeansDenoisingColored(sharpened, None, 10, 10, 7, 21)
    
    return denoised

In [36]:
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

In [37]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [38]:

K = len(set(y_train))

i = Input(shape=x_train[0].shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)

model = Model(i, x)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_12 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_13 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 16, 16, 32)       0   

In [39]:
model.compile(optimizer='adam',
			loss='sparse_categorical_crossentropy',
			metrics=['accuracy'])


In [40]:
with tf.device("gpu:0"):
    r = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=5)


Epoch 1/5
1563/1563 [==============================] - 19s 11ms/step - loss: 1.2888 - accuracy: 0.5521 - val_loss: 1.1591 - val_accuracy: 0.6070
Epoch 2/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8502 - accuracy: 0.7072 - val_loss: 0.7332 - val_accuracy: 0.7504
Epoch 3/5
1563/1563 [==============================] - 16s 11ms/step - loss: 0.6815 - accuracy: 0.7658 - val_loss: 0.6845 - val_accuracy: 0.7699
Epoch 4/5
1563/1563 [==============================] - 17s 11ms/step - loss: 0.5758 - accuracy: 0.8022 - val_loss: 0.7187 - val_accuracy: 0.7652
Epoch 5/5
1563/1563 [==============================] - 18s 12ms/step - loss: 0.4849 - accuracy: 0.8341 - val_loss: 0.7096 - val_accuracy: 0.7740


In [41]:
predictions = np.argmax(model.predict(x_test), axis=-1)
conf_matrix = confusion_matrix(y_test, predictions)
conf_matrix

313/313 [==============================] - 1s 3ms/step


array([[670,   8, 173,  41,  24,   7,  12,   7,  34,  24],
       [ 21, 895,   8,  10,   4,   5,  10,   1,  13,  33],
       [ 15,   0, 843,  31,  37,  33,  34,   5,   2,   0],
       [  4,   2, 117, 579,  36, 140, 108,  11,   1,   2],
       [  5,   0, 156,  29, 738,  23,  38,   9,   2,   0],
       [  2,   1,  79, 119,  28, 735,  31,   5,   0,   0],
       [  1,   0,  86,  39,   4,   9, 859,   1,   1,   0],
       [  6,   0,  52,  53,  69,  67,  14, 736,   1,   2],
       [ 36,  13,  35,  25,  12,   4,  11,   0, 852,  12],
       [ 19,  78,   7,  21,   2,   3,  14,   7,  16, 833]], dtype=int64)